## This file extracts data from the eo-patches to produce pandas dataframes suitable for machine learning
- The raw harvest data is not public and therefore not included in the repository

In [4]:
import pandas as pd
import json
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon, Point
import matplotlib.pyplot as plt
import pyproj
import gzip
import shutil
import glob
import os
import datetime
from tqdm import tqdm
import re
from functools import reduce
from operator import mul
from time import time
from enum import Enum, auto

from util_paths import SWEDEN
from util_paths import HARVEST_17_20_AOI_FILE
from util_paths import SENTINEL_1_11x11_SAR2SAR_V3_PATH
from util_paths import SHARED_FOLDER_PATH
from util_paths import UTM33N
from util_paths import find_eo_patches, extract_eo_patch_data

from utils_fvs_sampling_save import *

In [2]:
YEARS = [2017, 2018, 2019, 2020]
SENT_1 = SENTINEL_1_11x11_SAR2SAR_V3_PATH
SENTINEL_1_PATHS = {year: os.path.join(SENT_1, str(year)) for year in YEARS}

In [3]:
SENTINEL_1_PATHS

{2017: '/mimer/NOBACKUP/groups/snic2022-23-428/sentinel_1_data/despeckle_sar2sarV3_res_11m/2017',
 2018: '/mimer/NOBACKUP/groups/snic2022-23-428/sentinel_1_data/despeckle_sar2sarV3_res_11m/2018',
 2019: '/mimer/NOBACKUP/groups/snic2022-23-428/sentinel_1_data/despeckle_sar2sarV3_res_11m/2019',
 2020: '/mimer/NOBACKUP/groups/snic2022-23-428/sentinel_1_data/despeckle_sar2sarV3_res_11m/2020'}

In [4]:
#HARVEST_DATA_FILES = {year: os.path.join(SHARED_FOLDER_PATH, "hostvete", f"gridify_{year}_hv_mv.json") for year in YEARS}
#HARVEST_DATA_FILES = {year: os.path.join(SHARED_FOLDER_PATH, "hostvete", f"hostvete_weather_topology_augmented_{year}.json") for year in YEARS}
HARVEST_DATA_FILES = {
    year: os.path.join(SHARED_FOLDER_PATH, 
        "raw_data_12m", 
        f"weather_topology_augmented", 
        f"hostvete_weather_topology_augmented_{year}.json"
    ) 
        for year in YEARS
}
HARVEST_DATA_FILES

{2017: '/mimer/NOBACKUP/groups/snic2022-23-428/shared_oliver_christoffer/raw_data_12m/weather_topology_augmented/hostvete_weather_topology_augmented_2017.json',
 2018: '/mimer/NOBACKUP/groups/snic2022-23-428/shared_oliver_christoffer/raw_data_12m/weather_topology_augmented/hostvete_weather_topology_augmented_2018.json',
 2019: '/mimer/NOBACKUP/groups/snic2022-23-428/shared_oliver_christoffer/raw_data_12m/weather_topology_augmented/hostvete_weather_topology_augmented_2019.json',
 2020: '/mimer/NOBACKUP/groups/snic2022-23-428/shared_oliver_christoffer/raw_data_12m/weather_topology_augmented/hostvete_weather_topology_augmented_2020.json'}

In [ ]:
harvest_data = gpd.read_file(HARVEST_DATA_FILES[2017]).to_crs(crs = UTM33N)

# Class for a single EO patch

- Allows us to do basic checks and sample from a single patch
- Only basic Nearest neighbour sampling implemented which is probably sufficient for 50m resolution, radial sampling requires a more sophisticated method because of the low resolution in satellite imagery

In [5]:
class SampleType(Enum):
    NearestPixel = auto()
    Grid = auto()

In [6]:
class EoPatch:
    
    def __init__(self, year, patch_id, path):
        self.year = year
        self.patch_id = patch_id
        self.bbox = gpd.read_file(os.path.join(path, "bbox.geojson")).loc[0].geometry
        
        with open(os.path.join(path, "timestamp.json"), 'r') as f:
            self.dates = [
                datetime.datetime.strptime(ts, "%Y-%m-%dT%H:%M:%S") 
                    for ts in json.loads(f.read())
            ]
            
        with open(os.path.join(path, "data", "IW.npy"), 'rb') as f:
            # IW.npy = (time, longitude, latitude, [vh, vv])
            self.VH_idx = 0
            self.VV_idx = 1
            self.radar_tensor = np.load(f)
            
        self.n_rows = self.radar_tensor.shape[1] # y range
        self.n_cols = self.radar_tensor.shape[2] # x range
        
        #Verify that we have approximately the same resolution (meters/pixel) in both lat and lon
        res_x = (self.bbox.bounds[2] - self.bbox.bounds[0])/self.n_cols
        res_y = (self.bbox.bounds[3] - self.bbox.bounds[1])/self.n_rows
        if round(res_x,0) == round(res_y,0):
            self.res = round(res_x,0)
        else:
            print(f"Resolution in latitude and longitude for EoPatch({self.patch_id})does not match")
        
    def __str__(self):
        return f"EoPatch({self.patch_id})"

    def _dates_in_week(week_nbr):
        dates_in_week = [date for (date, w) in self.intersection_week if w == week]

    def _find_indices_for_dates(self, dates):
        return [i for (i, date_time) in enumerate(self.dates) if date_time.date() in dates]
    
    def contains(self, lon, lat):
        """Check if given point is inside patch."""
        return self.bbox.contains(Point(lon,lat))
    
    def coord_to_pixel(self, lon, lat):
        """Convert UTM33N coordinates to pixel coordinate in array (x,y)."""
        relative_lon = lon - self.bbox.bounds[0]
        relative_lat = lat - self.bbox.bounds[1]
        x = np.floor(relative_lon / self.res).astype(int)
        # Larger Lat -> Small Y, Small lat -> Large Y
        relative_y = (self.n_rows - 1) - np.floor(relative_lat / self.res).astype(int)
        y = max(relative_y, 0)
        
        return x, y
    
    def _get_neighbours(self, x, y, i):
        # clock wise, starting at 12
        # We just assume that no point is exactly on the border for this to work ...
        neighbours_idx = [
            (x, y),
            (x, y-1),   # north
            (x+1, y-1), # north east
            (x+1, y),   # east
            (x+1, y+1), # south east
            (x, y+1),   # south
            (x-1, y+1), # south west
            (x-1, y),   # west
            (x-1, y-1), # north west
        ]
        
        return np.array([self.radar_tensor[i, ny, nx, :] for nx, ny in neighbours_idx])
        
        
    def _sample(self,x,y,dates_in_week):
        for i in self._find_indices_for_dates(dates_in_week):
            # First version
            sample = self.radar_tensor[i,y,x,:]
            if not np.isnan(sample).any():
                return sample
                
        return np.array([np.nan,np.nan])
    
    def _sample_grid(self, x, y, dates_in_week):
        for i in self._find_indices_for_dates(dates_in_week):
            #including x, y point
            neighbours = self._get_neighbours(x,y,i)
            if not np.isnan(neighbours).any():
                return neighbours
            
        return np.full((9, 2), np.nan)
                
    #Use nearest neighbor sampling for now
    def sample(self, lon, lat, dates_in_week, sample_type):
        if not self.contains(lon,lat):
            print("Error: Point not inside patch bbox")
            return
        x, y = self.coord_to_pixel(lon,lat)
        
        if sample_type == SampleType.NearestPixel:
            return self._sample(x, y, dates_in_week)
        
        elif sample_type == SampleType.Grid:
            return self._sample_grid(x, y, dates_in_week)
        
        else:
            raise ValueError("Unknown sample type", sample_type)
    

# Class for all EO patches

- Allows us to sample from all patches

In [7]:
class EoPatches:
    
    def __init__(self, year, sent_1_path, sample_type=SampleType.NearestPixel):
        self.year = year
        patch_ids, paths = find_eo_patches(sent_1_path)
        self.patches = [
            EoPatch(year, patch_id, path) for patch_id, path in tqdm(zip(patch_ids, paths))
        ]
        self.sample_type = sample_type
        self.sync_dates()
        
    def sync_dates(self):
        self._find_intersecting_timestamps()
        self._filter_eo_patches_dates()
        self._check_contains_same_dates
        
        
    def sample(self, lon, lat, week):
        """Get the pixel value from Sentinel-1 SAR or None."""
        for patch in self.patches:
            if patch.contains(lon, lat):
                dates_in_week = [date for (date, w) in self.intersection_week if w == week]
                return patch.sample(lon,lat,dates_in_week, self.sample_type)
        # print(f"Error: Point {lat} {lon} is not contained in any Eo Patch")
    
    def _filter_eo_patches_dates(self):
        for patch in self.patches:
            indices = []
            for i, date in enumerate(patch.dates):
                # remove this data from patch
                if date.date() not in self.intersection:
                    del patch.dates[i]
                    indices.append(i)
            patch.radar_tensor = np.delete(patch.radar_tensor, indices, axis=0)  
    
    def _find_intersecting_timestamps(self):
        """Find common dates for all Eo Patches."""
        sets = [
            set([dt.date() for dt in patch.dates])
                for patch in self.patches
        ]
        self.intersection = list(set.intersection(*sets))
        self.intersection.sort()
        self.intersection_week = [
            (date, date.isocalendar().week)
                for date in self.intersection
        ]
        self.first_week = min(self.intersection).isocalendar().week
        self.last_week = max(self.intersection).isocalendar().week
        
    def _check_contains_same_dates(self):
        sets = [
            set([dt.date() for dt in patch.dates])
                for patch in self.patches
        ]
        diff = set.difference(*sets, set(self.intersection))
        print(diff)
            


## Indices using the VV and VH radar data

In [8]:
def RVI(VH, VV):
    """Index defined from study. Seems to be inverted for our fields.
    
    Always > 0
    """
    return VH / (VV+VH)

def my_RVI(VH, VV):
    """Our own rvi, as we see better results for this.
    
    Always > 0
    """
    return VV / (VV+VH)

def R_VV_VH(VH, VV):
    # Always > 0
    return VV / VH

def R_VH_VV(VH, VV):
    # Always > 0
    return VH / VV

def my_index0(VH, VV):
    # Can be negative
    return (VH-VV) / (VH+VV)

def my_index1(VH, VV):
    # Can be negative
    return (VH-VV) / VH

### Create Grid of 1 dist neighbors

In [9]:
def create_dataset_general(year, out_feather_name, folder, sample_type):
    path = os.path.join(folder, out_feather_name)
    if os.path.exists(path):
        print("Already created this dataset, skipping ...")
        return
    
    print(f"Loading eo patches, {year}")
    start = time()
    eo_patches = EoPatches(year, SENTINEL_1_PATHS[year], sample_type)
    end = time()
    print(f"Elapsed time: {end-start}")
    
    print("Loading harvest data")
    harvest_data = gpd.read_file(HARVEST_DATA_FILES[year]).to_crs(crs = UTM33N)
    weeks = range(eo_patches.first_week, eo_patches.last_week)
    columns = ["harvest", "year"]
    
    for week in weeks:
        if sample_type == SampleType.Grid:
            for n_name in neighbours_col_names:
                columns += [
                    f"{n_name}_vh_week_{week}",
                    f"{n_name}_vv_week_{week}",
                    f"{n_name}_rvi_week_{week}",
                    f"{n_name}_my_rvi_week_{week}",
                    f"{n_name}_vh/vv_week_{week}",
                    f"{n_name}_vv/vh_week_{week}",
                    f"{n_name}_mi0_week_{week}",
                    f"{n_name}_mi1_week_{week}"
                ]
                
        elif sample_type == SampleType.NearestPixel:
            columns += [
                f"vh_week_{week}",
                f"vv_week_{week}",
                f"rvi_week_{week}",
                f"my_rvi_week_{week}",
                f"vh/vv_week_{week}",
                f"vv/vh_week_{week}",
                f"mi0_week_{week}",
                f"mi1_week_{week}"
            ]
            
        else:
            raise ValueError("Unknown sample type", sample_type)
            
    
    def create_row(lon, lat, harvest):
        row = [harvest, year]
        
        for week in weeks:
            sample = eo_patches.sample(lon,lat,week) 
            if sample_type == SampleType.Grid:
                for neighbour in sample:
                    vh, vv = tuple(neighbour)
                    row += [vh, vv, RVI(vh, vv), my_RVI(vh, vv), R_VH_VV(vh, vv), R_VV_VH(vh, vv), my_index0(vh, vv), my_index1(vh, vv)]
                    
            elif sample_type == SampleType.NearestPixel:
                vh, vv = tuple(sample)
                row += [vh, vv, RVI(vh, vv), my_RVI(vh, vv), R_VH_VV(vh, vv), R_VV_VH(vh, vv), my_index0(vh, vv), my_index1(vh, vv)]
            
            else:
                raise ValueError("Unknown sample type", sample_type)
        return row
    
    print("Zipping data used form field")
    sample_data = zip(harvest_data.x.values, harvest_data.y.values, harvest_data.average_harvest)
    
    print("Creating dataframe")
    start = time()
    df = pd.DataFrame(
        [create_row(lon,lat,harvest) for (lon, lat, harvest) in tqdm(sample_data, position=0, leave=True)],
        columns=columns
    )
    end = time()
    print(f"Elapsed time: {end-start}")
    
    weather_df = harvest_data.loc[:, [col for col in harvest_data.columns if is_weather(col)]]
    if sample_type == SampleType.Grid:
        topology_df = harvest_data.loc[:, [col for col in harvest_data.columns if is_topology(col) or is_int_topology(col)]]
        height_df = harvest_data.loc[:, [col for col in harvest_data.columns if is_height(col)]]

    elif sample_type == SampleType.NearestPixel:
        topology_df = harvest_data.loc[:, [col for col in harvest_data.columns if (is_topology(col) or is_int_topology(col)) and is_nearest_sampling(col)]]
        height_df = harvest_data.loc[:, [col for col in harvest_data.columns if is_height(col) and is_nearest_sampling(col)]]

    else:
        raise ValueError("Unknown sample type", sample_type)
    
    df = pd.concat([df, weather_df, topology_df, height_df], axis=1)
    print(f"Drop NaN and save dataset at: {out_feather_name}")
    df = df.dropna().reset_index().drop(["index"], axis=1)
    
    df.to_feather(path)
    

In [ ]:
HARVEST_DATA_FILES

In [10]:
folder = "/mimer/NOBACKUP/groups/snic2022-23-428/shared_oliver_christoffer/datasets/12m"

def run_grid():    
    res = 11
    sample_type = SampleType.NearestPixel
    for year in YEARS:
        out_feather_name = f"dataset_y_{year}_res_{res}_sar2sar_v3.feather"
        create_dataset_general(year, out_feather_name, folder, sample_type)
        print()
        
run_grid()

Loading eo patches, 2017


6it [00:20,  3.34s/it]


Elapsed time: 33.09646725654602
Loading harvest data
Zipping data used form field
Creating dataframe


64842it [03:58, 271.86it/s]


Elapsed time: 241.50645446777344
Drop NaN and save dataset at: dataset_y_2017_res_11_sar2sar_v3.feather

Loading eo patches, 2018


...

Elapsed time: 347.92143964767456
Drop NaN and save dataset at: dataset_y_2019_res_11_sar2sar_v3.feather

Loading eo patches, 2020


6it [00:58,  9.69s/it]


Elapsed time: 72.97958397865295
Loading harvest data
Zipping data used form field
Creating dataframe


98528it [05:23, 305.04it/s]


Elapsed time: 327.5647096633911
Drop NaN and save dataset at: dataset_y_2020_res_11_sar2sar_v3.feather



In [11]:
def concat_years(out_feather_name, folder):
    paths = glob.glob(os.path.join(folder, '*.feather'))
    paths.sort()
    
    cols = [pd.read_feather(path).columns for path in paths]

    all_columns = set.union(*[set(col) for col in cols])
    shared_columns = set.intersection(*[set(col)for col in cols])
    not_shared_columns = list(set.difference(all_columns, shared_columns))
    
    #print(len(all_columns))
    #for c in sorted(list(all_columns)):
    #    print(c)
        
    dfs = []
    for path in paths:
        df = pd.read_feather(path)
        cols = df.columns
        df = df.drop(
            [col for col in cols if col in not_shared_columns],
            axis=1
        )
        dfs.append(df)
    
    df_all = pd.concat(dfs).dropna().reset_index().drop(["index"], axis=1)
    
    df_all.to_feather(os.path.join(SHARED_FOLDER_PATH, "datasets", out_feather_name))

    return df_all



In [12]:
df_all_name = f"dataset_y_{YEARS[0]}_{YEARS[-1]}_11m_hd.feather"
df_all = concat_years(df_all_name, folder)

In [ ]:
for col in sorted(df_all[df_all["year"] == 2020].columns):
    print(col)

In [ ]:
df_all[df_all["year"] == 2017]

In [15]:
for col in df_all.columns:
    print(col)

harvest
year
vh_week_14
vv_week_14
rvi_week_14
my_rvi_week_14
vh/vv_week_14
vv/vh_week_14
mi0_week_14
mi1_week_14
vh_week_15
vv_week_15
rvi_week_15
my_rvi_week_15
vh/vv_week_15
vv/vh_week_15
mi0_week_15
mi1_week_15
vh_week_16
vv_week_16
rvi_week_16
my_rvi_week_16
vh/vv_week_16
vv/vh_week_16
mi0_week_16
mi1_week_16
vh_week_17
vv_week_17
rvi_week_17
my_rvi_week_17
vh/vv_week_17
vv/vh_week_17
mi0_week_17
mi1_week_17
vh_week_18
vv_week_18
rvi_week_18
my_rvi_week_18
vh/vv_week_18
vv/vh_week_18
mi0_week_18
mi1_week_18
vh_week_19
vv_week_19
rvi_week_19
my_rvi_week_19
vh/vv_week_19
vv/vh_week_19
mi0_week_19
mi1_week_19
vh_week_20
vv_week_20
rvi_week_20
my_rvi_week_20
vh/vv_week_20
vv/vh_week_20
mi0_week_20
mi1_week_20
vh_week_21
vv_week_21
rvi_week_21
my_rvi_week_21
vh/vv_week_21
vv/vh_week_21
mi0_week_21
mi1_week_21
vh_week_22
vv_week_22
rvi_week_22
my_rvi_week_22
vh/vv_week_22
vv/vh_week_22
mi0_week_22
mi1_week_22
vh_week_23
vv_week_23
rvi_week_23
my_rvi_week_23
vh/vv_week_23
vv/vh_week_23
m

In [ ]:
df_all.loc[:, [col for col in df_all.columns if is_topology(col) or is_int_topology(col)]]

In [ ]:
dfh = df_all[[col for col in df_all.columns if is_height(col)]]

# Nearest pixel only

- We will construct a similair dataframe as Alexandros did with [harvest, vv_week_1, vh_week_1, ... ]

In [ ]:
def create_dataset(year, out_feather_name, folder, avg_neighbours: bool = False):
    path = os.path.join(folder, out_feather_name)
    if os.path.exists(path):
        print("Already created this dataset, skipping ...")
        return
    
    print(f"Loading eo patches, {year}")
    start = time()
    eo_patches = EoPatches(year, SENTINEL_1_PATHS[year])
    end = time()
    print(f"Elapsed time: {end-start}")
    
    print("Loading harvest data")
    harvest_data = gpd.read_file(HARVEST_DATA_FILES[year]).to_crs(crs = UTM33N)
    weeks = range(eo_patches.first_week, eo_patches.last_week)
    columns = ["harvest", "year"]
    for week in weeks:
        columns += [f"vh_week_{week}", f"vv_week_{week}", f"rvi_week_{week}", f"my_rvi_week_{week}", f"vh/vv_week_{week}", f"vv/vh_week_{week}", f"mi0_week_{week}", f"mi1_week_{week}"]
                      
    def create_row(lon, lat, harvest):
        row = [harvest, year]
        for week in weeks:
            vh, vv = tuple(eo_patches.sample(lon,lat,week,avg_neighbours))
            row += [vh, vv, RVI(vh, vv), my_RVI(vh, vv), R_VH_VV(vh, vv), R_VV_VH(vh, vv), my_index0(vh, vv), my_index1(vh, vv)]
        return row
    
    print("Zipping data used form field")
    sample_data = zip(harvest_data.x.values, harvest_data.y.values, harvest_data.average_harvest)
    
    print("Creating dataframe")
    start = time()
    df = pd.DataFrame(
        [create_row(lon,lat,harvest) for (lon, lat, harvest) in tqdm(sample_data, position=0, leave=True)],
        columns = columns
    )
    end = time()
    print(f"Elapsed time: {end-start}")
    
    
    print(f"Drop NaN and save dataset at: {out_feather_name}")
    df = df.dropna().reset_index().drop(["index"], axis=1)
    
    df.to_feather(path)
    

In [ ]:
def run():    
    res = 11
    folder = "/mimer/NOBACKUP/groups/snic2022-23-428/shared_oliver_christoffer/datasets/res11_sar2sar_v3_avg_neighbors"
    for year in YEARS:
        out_feather_name = f"dataset_y_{year}_res_{res}_sar2sar_v3.feather"
        create_dataset(year, out_feather_name, folder, True)
        print()
run()

In [ ]:
def concat_years(out_feather_name, folder):
    paths = glob.glob(os.path.join(folder, '*.feather'))
    paths.sort()
    
    cols = [pd.read_feather(path).columns for path in paths]

    all_columns = set.union(*[set(col) for col in cols])
    shared_columns = set.intersection(*[set(col)for col in cols])
    not_shared_columns = list(set.difference(all_columns, shared_columns))
    
    #print(len(all_columns))
    #for c in sorted(list(all_columns)):
    #    print(c)
        
    dfs = []
    for path in paths:
        df = pd.read_feather(path)
        cols = df.columns
        df = df.drop(
            [col for col in cols if col in not_shared_columns],
            axis=1
        )
        dfs.append(df)
    
    df_all = pd.concat(dfs).dropna().reset_index().drop(["index"], axis=1)
    
    df_all.to_feather(os.path.join(SHARED_FOLDER_PATH, "datasets", out_feather_name))

    return df_all



In [ ]:
df_all_name = f"dataset_y_{YEARS[0]}_{YEARS[-1]}_res_11_sar2sar_v3_avg_neighbors.feather"
folder = "/mimer/NOBACKUP/groups/snic2022-23-428/shared_oliver_christoffer/datasets/res11_sar2sar_v3_avg_neighbors"
df_all = concat_years(df_all_name, folder)


## Debugging coord to pixel

In [ ]:
patch_ids, paths = find_eo_patches(SENTINEL_1_PATHS[2019])

In [ ]:
patch = EoPatch(2019, patch_ids[0], paths[0])

In [ ]:
patch.bbox.bounds

In [ ]:
harvest_data = gpd.read_file(HARVEST_DATA_FILES[2019]).to_crs(crs = UTM33N)

In [ ]:
patch.radar_tensor.shape

In [ ]:
patch.patch_id

In [ ]:
vh0 = patch.radar_tensor[0, :,:, 0]

In [ ]:
vv0 = patch.radar_tensor[0, :,:, 1]

In [ ]:
fig, ax = plt.subplots()
ax.imshow(vh0, clim=(0, 150), cmap="gray")

fig, ax = plt.subplots()
ax.imshow(vv0, clim=(0, 150), cmap="gray")

In [ ]:
points = [(lon, lat) for lon, lat in zip(harvest_data.x, harvest_data.y) if patch.contains(lon, lat)]

In [ ]:
idx = [(x, y) for x, y in map(lambda p: patch.coord_to_pixel(p[0], p[1]), points)]

In [ ]:
fig, ax = plt.subplots()
ax.plot([x for x, _ in points], [y for _, y in points], '*')

fig, ax = plt.subplots()
ax.plot([x for x, _ in idx], [y for _, y in idx], '*')

print("This is correct, as a larger latitude should result in a small y")

In [ ]:
patch.bbox.bounds

In [ ]:
bot_left = patch.coord_to_pixel(round(patch.bbox.bounds[0]), round(patch.bbox.bounds[1]))

In [ ]:
bot_left

In [ ]:
top_right = patch.coord_to_pixel(round(patch.bbox.bounds[2]), round(patch.bbox.bounds[3]))

In [ ]:
top_right

In [ ]:
top_right_ish = patch.coord_to_pixel(round(patch.bbox.bounds[2]-300), round(patch.bbox.bounds[3]-300))

In [ ]:
top_right_ish

In [ ]:
patch.radar_tensor.shape

In [ ]:
fig, ax = plt.subplots()
ax.plot([x for x, _ in points], [y for _, y in points], '*')

fig, ax = plt.subplots()
ax.plot([x for x, _ in idx], [y for _, y in idx], '*')